In [1]:
import dgl
from dgl.data import DGLDataset
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import dgl.data
from dgl.nn import GraphConv,MaxPooling
import matplotlib.pyplot as plt
from tqdm import tqdm
import torch.optim as optim
import numpy as np
import time
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
import os

%matplotlib inline

In [2]:
class STL10TrainDataset(DGLDataset):
    def __init__(self,data_path,transforms=None):
        self.data_path = data_path
        self.transforms = transforms
        super().__init__(name='stl10_train_gprah')
    
    def process(self):
        GRAPHS, LABELS = dgl.load_graphs(self.data_path) #保存したグラーフデータの読み込み
        self.graphs = GRAPHS #グラフリストを代入
        self.labels = LABELS['label'] #ラベル辞書の値のみ代入
        self.dim_nfeats=len(self.graphs[0].ndata['feat'][0])

    def __getitem__(self, idx):
        if self.transforms == None:
            return self.graphs[idx], self.labels[idx]
        else:
            data=self.transforms(self.graphs[idx])
            return data,self.labels[idx]
    def __len__(self):
        return len(self.graphs)


class STL10TestDataset(DGLDataset):
    def __init__(self,data_path,transforms=None):
        self.data_path = data_path
        self.transforms = transforms
        super().__init__(name='stl10_test_gprah')
    
    def process(self):
        GRAPHS, LABELS = dgl.load_graphs(self.data_path) #保存したグラーフデータの読み込み
        self.graphs = GRAPHS #グラフリストを代入
        self.labels = LABELS['label'] #ラベル辞書の値のみ代入
        self.dim_nfeats=len(self.graphs[0].ndata['feat'][0])

    def __getitem__(self, idx):
        if self.transforms == None:
            return self.graphs[idx], self.labels[idx]
        else:
            data=self.transforms(self.graphs[idx])
            return data,self.labels[idx]
        
    def __len__(self):
        return len(self.graphs)

In [3]:
#transform = transforms.Compose([transforms.Normalize(0,1)])
traindataset=STL10TrainDataset('./data/STL10 Datasets/train/nnum100_ndatades_enone.dgl')
testdataset=STL10TestDataset('./data/STL10 Datasets/test/nnum100_ndatades_enone.dgl')

In [4]:
if os.name =='posix':
    num_workers = 2
else:
    num_workers = 0
num_workers = 0
traindataloader = GraphDataLoader(traindataset,batch_size = 1,shuffle = True,num_workers = num_workers,pin_memory = True)
testdataloader = GraphDataLoader(testdataset,batch_size = 1000,shuffle = True,num_workers = num_workers,pin_memory = True)
print(f'num_wokers = {num_workers}')
print(os.name)

num_wokers = 0
posix


In [5]:
gdata=0
label=0
for batchgraph,labels in traindataloader:
    gdata=batchgraph
    label = labels
    break


In [9]:
gdata.ndata['feat']

tensor([[ 32., 246.,   5.,  ..., 253., 255.,  62.],
        [ 33., 166.,  13.,  ..., 248., 255.,   0.],
        [ 90.,   9., 178.,  ..., 245.,  15.,  63.],
        ...,
        [128.,   6.,  54.,  ..., 255., 191.,  52.],
        [ 97., 248.,  12.,  ..., 121., 255.,   0.],
        [160.,  31.,   2.,  ..., 255., 207.,  62.]])

In [12]:
gc=GraphConv(61,10)
h=gc(gdata,gdata.ndata['feat'].float())

print(h)

tensor([[ 1.6966e+02, -5.9466e+01,  8.8345e+01,  1.4188e+02,  9.6904e+01,
         -3.5435e+02,  2.4193e+02,  4.3915e+00, -1.0325e+02,  2.5314e+02],
        [ 1.6850e+02, -5.6409e+01,  9.1516e+01,  1.4095e+02,  9.6848e+01,
         -3.5761e+02,  2.4394e+02,  2.3325e+00, -1.0506e+02,  2.5332e+02],
        [ 1.6747e+02, -5.4833e+01,  8.8258e+01,  1.3868e+02,  9.7792e+01,
         -3.5494e+02,  2.4205e+02,  1.5036e+00, -1.0341e+02,  2.5454e+02],
        [ 1.6689e+02, -5.6082e+01,  9.1444e+01,  1.3913e+02,  9.7666e+01,
         -3.5361e+02,  2.4074e+02, -3.5338e-01, -1.0495e+02,  2.5415e+02],
        [ 1.6707e+02, -5.6332e+01,  9.2627e+01,  1.3912e+02,  9.9693e+01,
         -3.5564e+02,  2.4198e+02,  1.6464e+00, -1.0327e+02,  2.5271e+02],
        [ 1.6743e+02, -5.4949e+01,  9.0994e+01,  1.3768e+02,  9.8565e+01,
         -3.5516e+02,  2.4146e+02,  2.1340e+00, -1.0429e+02,  2.5292e+02],
        [ 1.6727e+02, -5.6910e+01,  9.0684e+01,  1.3858e+02,  9.8293e+01,
         -3.5307e+02,  2.4231e+0

In [13]:
print(h.shape)

torch.Size([100, 10])


In [18]:
li=nn.Linear(10,5)
lj=nn.Linear(61,5)
i=li(h)
j=lj(gdata.ndata['feat'])
print(i.shape)

torch.Size([100, 5])


In [19]:
print(j.shape)
print(j)

torch.Size([100, 5])
tensor([[ 2.4269e+02,  4.9064e+01,  4.7246e+01,  3.4127e+01,  4.9423e+01],
        [ 1.7600e+02, -2.3179e+01, -5.8512e+00,  6.9248e+01, -2.3293e+01],
        [ 1.4904e+02,  1.8145e+01,  1.9258e+01, -1.6017e+02,  4.1110e+01],
        [ 1.8415e+02, -4.1818e+01,  5.8759e+01, -6.0178e+01,  7.9695e+01],
        [ 1.3930e+02, -5.6320e+01,  4.6889e+01, -1.4074e+02,  4.5051e+01],
        [ 1.3745e+02,  9.4207e+01,  4.6528e+00, -7.8430e+01,  1.3321e+01],
        [ 2.2407e+02,  8.9495e+00,  1.1509e+01, -1.1094e+02,  3.6897e+01],
        [ 8.6130e+01, -1.3833e+01,  4.8440e+01, -1.0409e+02,  5.8949e+01],
        [ 1.0601e+02, -9.0670e+00, -3.8722e+00, -8.7553e+01,  8.4050e+01],
        [ 1.4760e+02,  2.8576e+00, -4.2039e+01, -1.1775e+02,  8.0724e+01],
        [ 1.7269e+02,  4.4856e+01,  1.4756e+01,  2.2272e+01,  8.2706e+01],
        [ 2.1281e+02,  5.8887e+01, -1.0549e+02, -1.8863e+02,  5.7872e+01],
        [ 1.6862e+02,  7.4779e+00, -8.5249e+01, -1.1100e+02,  1.5054e+01],
    

: 

In [ ]:
class ligcn_block(nn.Module):
    def __init__(self,in_feat,h_feat):
        super(ligcn_block,self).__init__()
        self.liner1=nn.Linear(in_feat,h_feat)
        self.gcn=GraphConv(h_feat,h_feat)
        self.liner2=nn.Linear(h_feat,h_feat)
    
    def forward(self,g,in_feat):
        x=F.relu(self.liner1(in_feat))
        x=F.relu(self.gcn(g,x))
        x=F.relu(self.liner2(x))

        return x


class GCN_rc(nn.Module):
    def __init__(self,in_feat,h_feat,num_classes):
        super(GCN_rc,self).__init__()
        self.b1=ligcn_block(in_feat,61)
        self.b2=ligcn_block(61,30)
        self.b3=ligcn_block(30,num_classes)

    def forward(self,g,in_feat):
        x=in_feat

        h=F.relu(self.b1(g,in_feat))
        h=F.relu(self.b2(g,h))
        h=F.relu(self.b3(g,h))

        g.ndata['h'] = h

        return dgl.mean_nodes(g,'h')


In [ ]:
class GCN(nn.Module):
    def __init__(self,in_feat,h_feat,num_classes):
        super(GCN,self).__init__()
        self.gconv1=GraphConv(in_feat,h_feat)
        self.gconv2=GraphConv(h_feat,h_feat)

        self.gconv3=GraphConv(h_feat,num_classes)
        self.gconv4=GraphConv(h_feat,num_classes)

        self.dropout=nn.Dropout(0.2)

    def forward(self,g,in_feat):
        #x=in_feat.detach()

        h=F.relu(self.gconv1(g,in_feat))
        h=F.relu(self.gconv2(g,h))
        h=self.dropout(h)
    
        #h=h+x

        h=self.gconv3(g,h)
        #h=F.relu(self.gconv4(g,h))
        
        g.ndata['h'] = h
        return dgl.mean_nodes(g,'h')

In [ ]:
print(traindataset.dim_nfeats)
model=GCN(traindataset.dim_nfeats,20,10)
optimizer=optim.Adam(model.parameters(),lr=0.001)
device=torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model.to(device)
epochs=500

61
